In [308]:
import pandas as pd
import pyarrow as pa
import json

In [256]:
data = pd.read_csv('./flights-10k.csv', encoding='utf-8', dtype={'FL_DATE': 'str', 'ARR_TIME': 'str', 'DEP_TIME': 'str'})
data = data.dropna()

In [257]:
arr_time = data.FL_DATE + data.ARR_TIME.replace('2400', '0000')
data['ARRIVAL'] = pd.to_datetime(arr_time, format='%Y%m%d%H%M')

dep_time = data.FL_DATE + data.DEP_TIME.replace('2400', '0000')
data['DEPARTURE'] = pd.to_datetime(dep_time, format='%Y%m%d%H%M')

In [258]:
df = data.drop(['FL_DATE', 'ORIGIN', 'DEST', 'ARR_TIME', 'DEP_TIME'], axis=1).dropna()

In [259]:
df.head()

,DEP_DELAY,ARR_DELAY,AIR_TIME,DISTANCE,ARRIVAL,DEPARTURE
0,72.0,43.0,109.0,954,2016-12-01 23:59:00,2016-12-01 21:39:00
1,-4.0,-9.0,201.0,1437,2016-12-01 12:33:00,2016-12-01 09:51:00
2,90.0,92.0,191.0,1258,2016-12-01 16:12:00,2016-12-01 12:32:00
3,-4.0,-31.0,147.0,1258,2016-12-01 09:38:00,2016-12-01 06:56:00
4,-4.0,3.0,137.0,1012,2016-12-01 12:35:00,2016-12-01 08:51:00


In [279]:
from datetime import date, datetime

def json_serial(obj):
    return int(obj.strftime("%s"))

d = {}
for column in df.columns:
    d[column]=list(df[column])

with open('flights-10k.json', 'w') as f:
    json.dump(d, f, default=json_serial, separators=(',', ':'))

In [297]:
df.describe()

,DEP_DELAY,ARR_DELAY,AIR_TIME,DISTANCE
count,9987.000000,9987.000000,9987.000000,9987.000000
mean,3.032342,-3.362371,114.885852,830.275558
std,28.687506,30.442178,71.967636,601.210999
min,-32.000000,-73.000000,11.000000,31.000000
25%,-6.000000,-16.000000,62.000000,395.500000
50%,-3.000000,-9.000000,94.000000,651.000000
75%,1.000000,1.000000,146.000000,1067.000000
max,1056.000000,1048.000000,624.000000,4983.000000


In [298]:
df.dtypes

DEP_DELAY           float64
ARR_DELAY           float64
AIR_TIME            float64
DISTANCE              int64
ARRIVAL      datetime64[ns]
DEPARTURE    datetime64[ns]
dtype: object

In [292]:
df2 = df.astype({'DEP_DELAY': 'int16', 'ARR_DELAY': 'int16', 'AIR_TIME': 'int16', 'DISTANCE': 'int16'})
df2.head()

,DEP_DELAY,ARR_DELAY,AIR_TIME,DISTANCE,ARRIVAL,DEPARTURE
0,72,43,109,954,2016-12-01 23:59:00,2016-12-01 21:39:00
1,-4,-9,201,1437,2016-12-01 12:33:00,2016-12-01 09:51:00
2,90,92,191,1258,2016-12-01 16:12:00,2016-12-01 12:32:00
3,-4,-31,147,1258,2016-12-01 09:38:00,2016-12-01 06:56:00
4,-4,3,137,1012,2016-12-01 12:35:00,2016-12-01 08:51:00


In [315]:
table = pa.Table.from_pandas(df2)

In [294]:
writer = pa.RecordBatchFileWriter('flights-10k.arrow', table.schema)
writer.write(table)
writer.close()

In [299]:
!ls -lah

total 377864
drwxr-xr-x@ 14 domoritz  staff   448B Jun  8 17:27 .
drwxr-xr-x  25 domoritz  staff   800B Jun  8 15:05 ..
-rw-r--r--@  1 domoritz  staff   6.0K Nov 29  2017 .DS_Store
drwxr-xr-x   3 domoritz  staff    96B Jun  8 16:31 .ipynb_checkpoints
-rw-r--r--@  1 domoritz  staff    28M Nov 28  2017 564230852_T_ONTIME.csv
-rw-r--r--@  1 domoritz  staff   6.3M Nov 28  2017 564230852_T_ONTIME.zip
-rw-r--r--   1 domoritz  staff    15K Jun  8 17:27 convert.ipynb
-rw-r--r--   1 domoritz  staff   316K Jun  8 17:22 flights-10k.arrow
-rw-r--r--   1 domoritz  staff   400K Nov 29  2017 flights-10k.csv
-rw-r--r--   1 domoritz  staff   409K Jun  8 17:20 flights-10k.json
-rw-r--r--   1 domoritz  staff    31M Nov 29  2017 flights-1m.csv
-rw-r--r--   1 domoritz  staff   7.7M Feb 13 21:44 flights-200k.csv
-rw-r--r--   1 domoritz  staff    92M Nov 29  2017 flights-3m.csv
-rw-r--r--@  1 domoritz  staff    18M Feb 13 21:44 flights-500k.csv
